## WIP
This is a copy of notebook 1

## Predicting Item Surges
Preparing data set

in this case, we will use MSR (Material Service request) and Requisition data.
Purchase Order, Voucher Invoice transactions are not appropriate for this use case,
because PO, Vouchers are created only when Goods are recevied.
Raw user's request are most accurately captures at MSR and REQ level.

**please see:** to see meaningful results, increase sample size in Julia/Python notebook and create data at-least 5-10 million rows per data frame
GitHub doesn't allow voluminous data upload, hence, I uploaded only 1000 rows per dataframe

look at assets/sampleData/sampleData.ipynb file and change variable SampleSize,
then re-run / extract sample data CSVs to load into Graph.

## Supply Chain Graph

```@python
import pyTigerGraph as tg
hostName = "https://p2p.i.tgcloud.io"
userName = "amit"
password = "password"
graphName = "P2PSCM"
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password, graphname=graphName)

conn.gsql("ls")
conn.gsql('''USE GLOBAL
DROP ALL
''')

conn.gsql('''
USE GLOBAL

CREATE VERTEX UNSPSC (PRIMARY_ID Code INT, KeyTO STRING, PARENT_KEY STRING, Title STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX GUDID (PRIMARY_ID PrimaryDI STRING, publicDeviceRecordKey STRING, publicVersionStatus STRING, deviceRecordStatus STRING, publicVersionNumber STRING, publicVersionDate STRING, devicePublishDate STRING, deviceCommDistributionEndDate STRING, deviceCommDistributionStatus STRING, brandName STRING, versionModelNumber STRING, catalogNumber STRING, dunsNumber STRING, companyName STRING, deviceCount STRING, deviceDescription STRING, DMExempt STRING, premarketExempt STRING, deviceHCTP STRING, deviceKit STRING, deviceCombinationProduct STRING, singleUse STRING, lotBatch STRING, serialNumber STRING, manufacturingDate STRING, expirationDate STRING, donationIdNumber STRING, labeledContainsNRL STRING, labeledNoNRL STRING, MRISafetyStatus STRING, rx STRING, otc STRING, deviceSterile STRING, sterilizationPriorToUse STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX LOCATION_MASTER (PRIMARY_ID city STRING, city_ascii STRING, state_id STRING, state_name STRING, county_fips STRING, county_name STRING, lat STRING, lng STRING, population STRING, density STRING, source STRING, military STRING, incorporated STRING, timezone STRING, ranking STRING, zips STRING, id STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX ORG_MASTER (PRIMARY_ID ENTITY STRING, GROUPTOORG STRING, DEPARTMENT STRING, UNIT STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX MSR (PRIMARY_ID UNIT STRING, MSR_DATE STRING, FROM_UNIT STRING, TO_UNIT STRING, GUDID STRING, QTY STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX PO (PRIMARY_ID UNIT STRING, PO_DATE STRING, VENDOR STRING, GUDID STRING, QTY STRING, UNIT_PRICE STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX SALES (PRIMARY_ID UNIT STRING, SALES_DATE STRING, STATUS STRING, SALES_RECEIPT_NUM STRING, CUSTOMER STRING, GUDID STRING, QTY STRING, UNIT_PRICE STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX SHIPRECEIPT (PRIMARY_ID UNIT STRING, SHIP_DATE STRING, STATUS STRING, SHIPMENT_NUM STRING, CUSTOMER STRING, GUDID STRING, QTY STRING, UNIT_PRICE STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX VENDOR (PRIMARY_ID brandName STRING, dunsNumber STRING, companyName STRING, rx STRING, otc STRING) WITH primary_id_as_attribute="true"

CREATE VERTEX VOUCHER (PRIMARY_ID UNIT STRING, VCHR_DATE STRING, STATUS STRING, VENDOR_INVOICE_NUM STRING, VENDOR STRING, GUDID STRING, QTY STRING, UNIT_PRICE STRING) WITH primary_id_as_attribute="true"

CREATE DIRECTED EDGE by_UNSPSC (From GUDID, To UNSPSC)
CREATE DIRECTED EDGE PO_LOCATION_MASTER (From PO, To LOCATION_MASTER)
CREATE DIRECTED EDGE PO_ORG_MASTER (From PO, To ORG_MASTER)
CREATE DIRECTED EDGE PO_VENDOR (From PO, To VENDOR)
CREATE DIRECTED EDGE PO_GUDID (From PO, To GUDID)
CREATE DIRECTED EDGE MSR_LOCATION_MASTER (From MSR, To LOCATION_MASTER)
CREATE DIRECTED EDGE MSR_ORG_MASTER (From MSR, To ORG_MASTER)
CREATE DIRECTED EDGE MSR_VENDOR (From MSR, To VENDOR)
CREATE DIRECTED EDGE MSR_GUDID (From MSR, To GUDID)
CREATE DIRECTED EDGE SALES_LOCATION_MASTER (From SALES, To LOCATION_MASTER)
CREATE DIRECTED EDGE SALES_ORG_MASTER (From SALES, To ORG_MASTER)
CREATE DIRECTED EDGE SALES_VENDOR (From SALES, To VENDOR)
CREATE DIRECTED EDGE SALES_GUDID (From SALES, To GUDID)
CREATE DIRECTED EDGE SHIPRECEIPT_LOCATION_MASTER (From SHIPRECEIPT, To LOCATION_MASTER)
CREATE DIRECTED EDGE SHIPRECEIPT_ORG_MASTER (From SHIPRECEIPT, To ORG_MASTER)
CREATE DIRECTED EDGE SHIPRECEIPT_VENDOR (From SHIPRECEIPT, To VENDOR)
CREATE DIRECTED EDGE SHIPRECEIPT_GUDID (From SHIPRECEIPT, To GUDID)
CREATE DIRECTED EDGE VOUCHER_LOCATION_MASTER (From VOUCHER, To LOCATION_MASTER)
CREATE DIRECTED EDGE VOUCHER_ORG_MASTER (From VOUCHER, To ORG_MASTER)
CREATE DIRECTED EDGE VOUCHER_VENDOR (From VOUCHER, To VENDOR)
CREATE DIRECTED EDGE VOUCHER_GUDID (From VOUCHER, To GUDID)
''')
results = conn.gsql('''CREATE GRAPH P2PSCM(UNSPSC, GUDID, LOCATION_MASTER, ORG_MASTER, MSR, PO, SALES, SHIPRECEIPT, VENDOR, VOUCHER, 
by_UNSPSC, PO_LOCATION_MASTER, PO_ORG_MASTER, PO_VENDOR, PO_GUDID, MSR_LOCATION_MASTER, MSR_ORG_MASTER, MSR_VENDOR, MSR_GUDID, SALES_LOCATION_MASTER, SALES_ORG_MASTER, SALES_VENDOR, SALES_GUDID, SHIPRECEIPT_LOCATION_MASTER, SHIPRECEIPT_ORG_MASTER, SHIPRECEIPT_VENDOR, SHIPRECEIPT_GUDID, VOUCHER_LOCATION_MASTER, VOUCHER_ORG_MASTER, VOUCHER_VENDOR, VOUCHER_GUDID)''')
```

**Do NOT forget to publish your data mappings and load data.**

![P2P Graph 4](https://github.com/AmitXShukla/P2P.ai/blob/main/docs/assets/images/graphp2p4.png?raw=true)

## local clustering coefficient

MSQ data, included GUDID orders by Unit and Requested date/
behavior we have seen is, most of GUDID "tied together" are bought together.
so, we will train, our clustering coefficeint algorithm, on GUDID "groups" by date,
and then later will predict, if any such GUDID falls in to which category.

then after, once we can run a simple query of GUDIDs transactions, where GUDID order is larger than average
and then, take those GUDID each, find out other GUDID which in same cluster.

this way, we will can predict, what other items shows surge/increase demand.


MSR data:
UNIT,MSR_DATE,FROM_UNIT,TO_UNIT,GUDID,QTY

tg_lcc(STRING v_type, STRING e_type,INT top_k=100,

BOOL print_accum = True, STRING result_attr = "",

STRING file_path = "", BOOL display_edges = FALSE)

RUN QUERY tg_lcc("UNIT","GUDID",_,_,_,_,_,_,_,_)